# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Identify-the-transcription-factors-in-the-hypoxia-response" data-toc-modified-id="Identify-the-transcription-factors-in-the-hypoxia-response-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Identify the transcription factors in the hypoxia response</a></div>

In this notebook, I dissect what transcription factors are present in the hypoxia response we defined earlier. 

In [1]:
# important stuff:
import pandas as pd

import tissue_enrichment_analysis as tea
import morgan as morgan
import epistasis as epi
import genpy

# Graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc

rc('text', usetex=True)
rc('text.latex', preamble=r'\usepackage{cmbright}')
rc('font', **{'family': 'sans-serif', 'sans-serif': ['Helvetica']})

# Magic function to make matplotlib inline;
%matplotlib inline

# This enables SVG graphics inline. 
# There is a bug, so uncomment if it works.
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style("dark")

ft = 35 #title fontsize
import gvars

mpl.rcParams['xtick.labelsize'] = 16 
mpl.rcParams['ytick.labelsize'] = 16 
mpl.rcParams['legend.fontsize'] = 14

In [2]:
tfs = pd.read_csv('../input/tf_list.csv')

In [3]:
q = 0.1
# this loads all the labels we need
genvar = gvars.genvars()

In [42]:
# extract the hypoxia response:
hyp_response = pd.read_csv('../output/hypoxia_response.csv')

# Identify the transcription factors in the hypoxia response

Let's extract them, and plot them.

In [31]:
hyp_response.head(7)

,index,genotype,target_id,ens_gene,ext_gene,b,se_b,qval,regulation,code,order
0,222,egl-9,B0286.5,WBGene00001438,fkh-6,1.219435,0.254926,0.000046,candidate for direct regulation,b,1
1,221,rhy-1,B0286.5,WBGene00001438,fkh-6,1.118830,0.254926,0.000241,candidate for direct regulation,e,1
2,219,vhl-1,B0286.5,WBGene00001438,fkh-6,0.610422,0.254926,0.183319,candidate for direct regulation,d,1
3,218,hif-1,B0286.5,WBGene00001438,fkh-6,0.478411,0.254926,0.371657,candidate for direct regulation,c,1
4,217,fog-2,B0286.5,WBGene00001438,fkh-6,-0.394214,0.360520,0.673249,candidate for indirect regulation,g,1
5,223,egl-9 hif-1,B0286.5,WBGene00001438,fkh-6,0.475106,0.254926,0.478144,candidate for direct regulation,f,1
6,220,egl-9;vhl-1,B0286.5,WBGene00001438,fkh-6,0.850703,0.254926,0.008202,candidate for direct regulation,a,1


In [67]:
# find tfs in the hif-1 response
tf_hif = hyp_response[hyp_response.target_id.isin(tfs.target_id)]
print('There are {0} transcription factors in HIF-1+ animals'.format(tfs_in_hif.shape[0]))

There are 15 transcription factors in HIF-1+ animals


In [51]:
tf_hif

,genotype,target_id,ens_gene,ext_gene,b,se_b,qval,regulation,code
217,fog-2,B0286.5,WBGene00001438,fkh-6,-0.394214,0.360520,6.732494e-01,candidate for indirect regulation,g
218,hif-1,B0286.5,WBGene00001438,fkh-6,0.478411,0.254926,3.716565e-01,candidate for direct regulation,c
219,vhl-1,B0286.5,WBGene00001438,fkh-6,0.610422,0.254926,1.833186e-01,candidate for direct regulation,d
220,egl-9;vhl-1,B0286.5,WBGene00001438,fkh-6,0.850703,0.254926,8.201514e-03,candidate for direct regulation,a
221,rhy-1,B0286.5,WBGene00001438,fkh-6,1.118830,0.254926,2.411036e-04,candidate for direct regulation,e
222,egl-9,B0286.5,WBGene00001438,fkh-6,1.219435,0.254926,4.559927e-05,candidate for direct regulation,b
223,egl-9 hif-1,B0286.5,WBGene00001438,fkh-6,0.475106,0.254926,4.781443e-01,candidate for direct regulation,f
399,fog-2,C02F12.5,WBGene00015355,C02F12.5,-0.937367,0.207967,1.538669e-04,candidate for indirect regulation,g
400,egl-9,C02F12.5,WBGene00015355,C02F12.5,1.460227,0.147055,5.288281e-21,candidate for direct regulation,b
401,rhy-1,C02F12.5,WBGene00015355,C02F12.5,1.429037,0.147055,6.954610e-20,candidate for direct regulation,e


In [56]:
tissue = tea.fetch_dictionary('tissue')
pheno = tea.fetch_dictionary('phenotype')
go = tea.fetch_dictionary('go')